In [1]:
import tensorflow as tf
from tensorflow import keras
from loguru import logger
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

2022-10-27 18:32:52.142595: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print (x_train.shape)
print (x_test.shape)

4423680/4422102 [==============================] - 1s 0us/step
(60000, 28, 28)
(10000, 28, 28)


In [4]:

encoder_input = keras.layers.Input(shape=(28,28))
